In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
url = "https://adindex.ru/ratings/marketing/2021/300770/"

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [4]:
dfs = []

In [8]:
with pd.ExcelWriter('data.xlsx') as writer:
  
    #Поиск названия таблицы
    h2s = soup.find_all('h2', class_='rating-subtitle')
    
    #Поиск таблицы
    tables = soup.find_all('table', class_='rating-table-full js-rating-datatable-table')
    
    #цикл zip для правильной валидации данных
    for h2, table in zip(h2s, tables):

        #Название листа в Excel вмещает в себя 31 символ
        sheet_name = h2.text[:31]

        rows = table.find_all('tr')
        headers = [th.text for th in rows[0].find_all('th')]

        data = []
        for row in rows[1:]:
            cols = row.find_all('td')
            row_data = [col.text for col in cols]
            data.append(row_data)

        df = pd.DataFrame(data, columns=headers)

        for df in dfs: #Сортировка по целому бюджету

            if "Открытая отчетность за 2021 год. Банки" in sheet_name:

                if {'2016', '2017', '2018', '2019', '2020', '2021'}.issubset(df.columns):
                    mean_budget = df[['2016', '2017', '2018', '2019', '2020', '2021']].mean(axis=1)
                    df['mean_budget'] = mean_budget

                    df = df.sort_values(by='mean_budget', ascending=False)
                    df = df.drop(columns=['mean_budget'])

        numeric_cols = df.select_dtypes(include=[np.number]).columns

        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce') #Приведения числа к Float
        df = df.replace('N/a', 'None') # Замена N/a на None
        df.to_excel(writer, sheet_name=sheet_name, index=False) # Запись в Excel


In [ ]:
print("Готово!")